For one frame we have F0=1/T0  (Hz)

We compute the mean of F0 of all frames and their standard deviation.

In [3]:
from pathlib import Path

import numpy as np
import pandas as pd
import parselmouth


# =========================================
# Your paths (as you gave them)
# =========================================
AUDIO_DIR = Path(r"C:\Users\user\Downloads\23849127\HC_AH\HC_AH")
OUTPUT_CSV = "F0_features.csv"


def compute_f0_stats_sustained(file_path,
                               time_step=0.01,
                               pitch_floor=75,
                               pitch_ceiling=300):
    """
    Compute F0_mean_sustained and F0_std_sustained from a sustained /a/ recording.

    Parameters
    ----------
    file_path : str or Path
        Path to the WAV file (sustained /a/).
    time_step : float
        Time step between pitch estimates (in seconds).
    pitch_floor : float
        Minimum expected F0 in Hz (e.g. 75 Hz).
    pitch_ceiling : float
        Maximum expected F0 in Hz (e.g. 300 Hz).

    Returns
    -------
    f0_mean : float
        Mean F0 over voiced frames (Hz).
    f0_std : float
        Standard deviation of F0 over voiced frames (Hz).
    """
    snd = parselmouth.Sound(str(file_path))

    # Praat pitch extraction
    pitch = snd.to_pitch(time_step=time_step,
                         pitch_floor=pitch_floor,
                         pitch_ceiling=pitch_ceiling)

    # F0 values (Hz)
    f0_values = pitch.selected_array["frequency"]  # shape: (num_frames,)

    # Only voiced frames (F0 > 0)
    f0_voiced = f0_values[f0_values > 0]

    if len(f0_voiced) == 0:
        return np.nan, np.nan

    f0_mean = float(np.mean(f0_voiced))
    f0_std = float(np.std(f0_voiced, ddof=0))  # population std

    return f0_mean, f0_std


def main():
    # Collect all .wav files in the given folder (non-recursive)
    wav_paths = sorted(AUDIO_DIR.glob("*.wav"))

    if not wav_paths:
        print(f"No .wav files found in: {AUDIO_DIR}")
        return

    rows = []

    for wav_path in wav_paths:
        filename = wav_path.name
        print(f"Processing: {filename}")

        try:
            f0_mean, f0_std = compute_f0_stats_sustained(wav_path)
        except Exception as e:
            print(f"  Error processing {filename}: {e}")
            f0_mean, f0_std = np.nan, np.nan

        rows.append({
            "filename": filename,
            "F0_mean_sustained_Hz": f0_mean,
            "F0_std_sustained_Hz": f0_std,
        })

    df = pd.DataFrame(rows)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\nDone. Saved CSV to: {OUTPUT_CSV}")


if __name__ == "__main__":
    main()


Processing: AH_064F_7AB034C9-72E4-438B-A9B3-AD7FDA1596C5.wav
Processing: AH_114S_A89F3548-0B61-4770-B800-2E26AB3908B6.wav
Processing: AH_121A_BD5BA248-E807-4CB9-8B53-47E7FFE5F8E2.wav
Processing: AH_123G_559F0706-2238-447C-BA39-DB5933BA619D.wav
Processing: AH_195B_39DA6A45-F4CC-492A-80D4-FB79049ACC22.wav
Processing: AH_197T_7552379A-2310-46E1-9466-9D8045C990B8.wav
Processing: AH_222K_FC9D2763-1836-460B-954F-37F23D6CD81D.wav
Processing: AH_264Z_593C20CD-0A54-4177-B031-26EE147080A3.wav
Processing: AH_292J_201CB911-31C1-4CD0-BD73-4FBA4A16C21F.wav
Processing: AH_322A_C3BF5535-A11E-498E-94EB-BE7E74099FFB.wav
Processing: AH_325A_3EB21DC7-C340-4D0E-AC9E-0EABF217BBEE.wav
Processing: AH_325J_7F5F27AA-5A93-43CF-AB17-FC53940BF4B0.wav
Processing: AH_333L_6C551A6E-CC47-410E-AA49-2DC0A86E6489.wav
Processing: AH_378G_3C2A05CE-36E4-4956-8FC2-0494B27D3EA8.wav
Processing: AH_420J_07C96C2C-6E96-4A2F-BEC9-5CB71DB309B6.wav
Processing: AH_444B_E1586F09-1BF5-408D-A55E-96D9E8B76A43.wav
Processing: AH_456K_CBF6

Intensity

Intensity is the sound pressure level in decibels(dB).

We compute the intensity for each frame and then take the mean and the standard deviation of the intensity values we have.

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd
import parselmouth


# =========================================
# Your paths (given by you)
# =========================================
AUDIO_DIR = Path(r"C:\Users\user\Downloads\23849127\HC_AH\HC_AH")
OUTPUT_CSV = "Intensity_features.csv"


def compute_intensity_stats_sustained(file_path,
                                      time_step=0.01,
                                      minimum_pitch=75):
    """
    Compute Intensity_mean_sustained and Intensity_std_sustained
    from a sustained /a/ recording.

    Parameters
    ----------
    file_path : str or Path
        Path to the WAV file (sustained /a/).
    time_step : float
        Time step between intensity estimates (in seconds).
    minimum_pitch : float
        Minimum pitch in Hz used by Praat to set the analysis window.

    Returns
    -------
    intensity_mean : float
        Mean intensity over frames (dB).
    intensity_std : float
        Standard deviation of intensity over frames (dB).
    """
    snd = parselmouth.Sound(str(file_path))

    # Praat intensity extraction (in dB)
    intensity = snd.to_intensity(time_step=time_step,
                                 minimum_pitch=minimum_pitch)

    # Intensity values is usually shape (1, num_frames)
    intensity_values = intensity.values.T.flatten()  # -> (num_frames,)

    # Keep only finite values
    valid = np.isfinite(intensity_values)
    intensity_valid = intensity_values[valid]

    if len(intensity_valid) == 0:
        return np.nan, np.nan

    intensity_mean = float(np.mean(intensity_valid))
    intensity_std = float(np.std(intensity_valid, ddof=0))

    return intensity_mean, intensity_std


def main():
    # Collect all .wav files in the given folder (non-recursive)
    wav_paths = sorted(AUDIO_DIR.glob("*.wav"))

    if not wav_paths:
        print(f"No .wav files found in: {AUDIO_DIR}")
        return

    rows = []

    for wav_path in wav_paths:
        filename = wav_path.name
        print(f"Processing: {filename}")

        try:
            intensity_mean, intensity_std = compute_intensity_stats_sustained(wav_path)
        except Exception as e:
            print(f"  Error processing {filename}: {e}")
            intensity_mean, intensity_std = np.nan, np.nan

        rows.append({
            "filename": filename,
            "Intensity_mean_sustained_dB": intensity_mean,
            "Intensity_std_sustained_dB": intensity_std,
        })

    df = pd.DataFrame(rows)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\nDone. Saved CSV to: {OUTPUT_CSV}")


if __name__ == "__main__":
    main()




Processing: AH_064F_7AB034C9-72E4-438B-A9B3-AD7FDA1596C5.wav
Processing: AH_114S_A89F3548-0B61-4770-B800-2E26AB3908B6.wav
Processing: AH_121A_BD5BA248-E807-4CB9-8B53-47E7FFE5F8E2.wav
Processing: AH_123G_559F0706-2238-447C-BA39-DB5933BA619D.wav
Processing: AH_195B_39DA6A45-F4CC-492A-80D4-FB79049ACC22.wav
Processing: AH_197T_7552379A-2310-46E1-9466-9D8045C990B8.wav
Processing: AH_222K_FC9D2763-1836-460B-954F-37F23D6CD81D.wav
Processing: AH_264Z_593C20CD-0A54-4177-B031-26EE147080A3.wav
Processing: AH_292J_201CB911-31C1-4CD0-BD73-4FBA4A16C21F.wav
Processing: AH_322A_C3BF5535-A11E-498E-94EB-BE7E74099FFB.wav
Processing: AH_325A_3EB21DC7-C340-4D0E-AC9E-0EABF217BBEE.wav
Processing: AH_325J_7F5F27AA-5A93-43CF-AB17-FC53940BF4B0.wav
Processing: AH_333L_6C551A6E-CC47-410E-AA49-2DC0A86E6489.wav
Processing: AH_378G_3C2A05CE-36E4-4956-8FC2-0494B27D3EA8.wav
Processing: AH_420J_07C96C2C-6E96-4A2F-BEC9-5CB71DB309B6.wav
Processing: AH_444B_E1586F09-1BF5-408D-A55E-96D9E8B76A43.wav
Processing: AH_456K_CBF6

RPDE:

Recurrence Period Density Entropy over the sustained /a/.
It measures how irregular and unpredictable the vibration pattern is.
Closer to 0 → more regular; closer to 1 → more complex / irregular.

DFA_alpha:

Scaling exponent (α) from Detrended Fluctuation Analysis on the signal.
It measures long-range correlations in the waveform (fractal-like behavior).
Different α values reflect different “texture” of the voice dynamics.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import parselmouth


# =========================================
# Your paths
# =========================================
AUDIO_DIR = Path(r"C:\Users\user\Downloads\23849127\HC_AH\HC_AH")
OUTPUT_CSV = "RPDE/DFA_features.csv"


def compute_rpde(signal, sr, m=3, tau_sec=0.01, r=0.3, max_T=200):
    """
    Compute a simple version of Recurrence Period Density Entropy (RPDE).

    Parameters
    ----------
    signal : 1D np.ndarray
        Audio samples (mono).
    sr : float
        Sampling rate (Hz).
    m : int
        Embedding dimension.
    tau_sec : float
        Embedding delay in seconds.
    r : float
        Recurrence radius in (normalized) state space.
    max_T : int
        Maximum recurrence period (in frames) to consider.

    Returns
    -------
    rpde : float
        Normalized entropy in [0, 1] (higher => more irregular),
        or NaN if it cannot be computed.
    """
    x = np.array(signal, dtype=float)

    if x.size < 2:
        return np.nan

    # Normalize
    x = x - np.mean(x)
    std = np.std(x)
    if std == 0:
        return np.nan
    x = x / std

    tau = max(1, int(tau_sec * sr))
    N = x.size - (m - 1) * tau
    if N <= 1:
        return np.nan

    # Time-delay embedding
    emb = np.empty((N, m), dtype=float)
    for i in range(N):
        emb[i] = x[i:i + m * tau:tau]

    counts = np.zeros(max_T, dtype=float)

    # For each point, look for first recurrence within radius r
    for i in range(N - 1):
        # Distances to future points
        diffs = emb[i + 1:] - emb[i]
        dists = np.linalg.norm(diffs, axis=1)

        rec_idx = np.where(dists < r)[0]
        if rec_idx.size == 0:
            continue

        # First recurrence time (in "frames" of the embedded series)
        T = rec_idx[0] + 1
        if 1 <= T < max_T:
            counts[T] += 1

    total = counts.sum()
    if total == 0:
        return np.nan

    p = counts / total
    p = p[p > 0]

    if p.size == 0:
        return np.nan

    H = -np.sum(p * np.log(p))
    H_norm = H / np.log(p.size)  # normalized to [0,1]

    return float(H_norm)


def compute_dfa_alpha(signal, min_window=10, max_window=None, num_windows=20):
    """
    Compute DFA scaling exponent alpha for a 1D signal.

    Parameters
    ----------
    signal : 1D np.ndarray
        Audio samples (mono).
    min_window : int
        Minimum window size (samples).
    max_window : int or None
        Maximum window size (samples). If None, use N//4.
    num_windows : int
        Number of window sizes (log-spaced).

    Returns
    -------
    alpha : float
        DFA scaling exponent, or NaN if it cannot be computed.
    """
    x = np.array(signal, dtype=float)
    N = x.size
    if N < 2:
        return np.nan

    x = x - np.mean(x)
    y = np.cumsum(x)

    if max_window is None:
        max_window = N // 4
    if max_window <= min_window:
        return np.nan

    # Log-spaced window sizes
    window_sizes = np.unique(
        np.floor(
            np.logspace(np.log10(min_window), np.log10(max_window), num_windows)
        ).astype(int)
    )

    F = []
    valid_sizes = []

    for s in window_sizes:
        if s < 2:
            continue
        n_segments = N // s
        if n_segments < 2:
            continue

        # Reshape into segments
        y_seg = y[:n_segments * s].reshape(n_segments, s)
        t = np.arange(s)
        rms_list = []

        for seg in y_seg:
            coeffs = np.polyfit(t, seg, 1)
            trend = np.polyval(coeffs, t)
            detrended = seg - trend
            rms = np.sqrt(np.mean(detrended ** 2))
            rms_list.append(rms)

        F_s = np.mean(rms_list)
        if F_s > 0:
            F.append(F_s)
            valid_sizes.append(s)

    if len(F) < 2:
        return np.nan

    log_s = np.log(valid_sizes)
    log_F = np.log(F)

    alpha, _ = np.polyfit(log_s, log_F, 1)
    return float(alpha)


def compute_nonlinear_features_sustained(file_path):
    """
    Compute RPDE_sustained and DFA_alpha_sustained for a sustained /a/ file.

    Returns
    -------
    rpde : float
    dfa_alpha : float
    """
    snd = parselmouth.Sound(str(file_path))
    samples = snd.values[0]  # mono
    sr = snd.sampling_frequency

    # Downsample to ~4 kHz to reduce cost for RPDE/DFA
    target_sr = 4000.0
    factor = max(1, int(sr // target_sr))
    samples_ds = samples[::factor]
    sr_ds = sr / factor

    if samples_ds.size < 100:
        return np.nan, np.nan

    rpde = compute_rpde(samples_ds, sr_ds)
    dfa_alpha = compute_dfa_alpha(samples_ds)

    return rpde, dfa_alpha


def main():
    # Collect all .wav files in the given folder (non-recursive)
    wav_paths = sorted(AUDIO_DIR.glob("*.wav"))

    if not wav_paths:
        print(f"No .wav files found in: {AUDIO_DIR}")
        return

    rows = []

    for wav_path in wav_paths:
        filename = wav_path.name
        print(f"Processing: {filename}")

        try:
            rpde, dfa_alpha = compute_nonlinear_features_sustained(wav_path)
        except Exception as e:
            print(f"  Error processing {filename}: {e}")
            rpde, dfa_alpha = np.nan, np.nan

        rows.append({
            "filename": filename,
            "RPDE_sustained": rpde,
            "DFA_alpha_sustained": dfa_alpha,
        })

    df = pd.DataFrame(rows)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\nDone. Saved CSV to: {OUTPUT_CSV}")


if __name__ == "__main__":
    main()


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Reading text recordings dataset required.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

We will compute F0 now but for the reading a text recordings dataset.